# Project 2

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
from joblib import dump, load
import re
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import sklearn
print(sklearn.__version__)

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix

0.20.2


ImportError: cannot import name 'plot_confusion_matrix'

## Preliminary analysis on collected local data 

In this section, we wanted to investigate what type of model (Random Forest, SVM, or Lostical Regression) would be the most promising. Additionally, we also wanted to find out what packet protocols should we train on given the relaxed conditions. This section does relate to the Bonus task. 

In [ ]:
path = '../../p2/collection'
col_names = ["website-index", "time", "direction", "packet size"]
folders = []
for r, d, f in os.walk(path):
    for folder in d:
        folders.append(folder)

results = {}

# Loop over the folders in /collection
for folder in folders:
    if (folder == "raw"):
        continue 
    results[folder] = {} # Goddam python dictionaries

    # Load data into dataframes
    df_one = pd.read_csv(path+"/"+folder+"/"+"Day1-parsed-ondevice.csv", header=0, names=col_names, error_bad_lines=False, warn_bad_lines=False, quotechar='^')
    df_two = pd.read_csv(path+"/"+folder+"/"+"Day2-parsed-ondevice.csv", header=0, names=col_names, error_bad_lines=False, warn_bad_lines=False, quotechar='^')
    df_three = pd.read_csv(path+"/"+folder+"/"+"Day3-parsed-ondevice.csv", header=0, names=col_names, error_bad_lines=False, warn_bad_lines=False, quotechar='^')
    
    # Preprocess training data
    df = df_one.append(df_two)
    X = df.loc[:, df.columns != 'website-index']
    y = df['website-index']
    
    # Preprocess testing data
    X_test = df_three.loc[:, df_three.columns != 'website-index']
    y_test = df_three['website-index']
    
    outpath = './models/' + folder + "/"
    if not os.path.exists(outpath):
        os.mkdir(outpath)    

    # RF model training
    clf = RandomForestClassifier(n_jobs=-1)
    clf.fit(X,y)
    rf_acc = clf.score(X_test, y_test)
    print("RF acc: " + str(rf_acc))
    results[folder]["rf"] = str(rf_acc)
    dump(clf, outpath + 'rf_model.joblib')
    print(results)

    # SVM model training
    svm_model = svm.SVC(verbose=True, cache_size=8000) # Yes i have the ram
    svm_model.fit(X, y)
    svm_acc = svm_model.score(X_test, y_test)
    print("SVM acc: " + str(svm_acc))
    results[folder]["svm"] = str(svm_acc)
    dump(svm_model, outpath + 'svm_model.joblib')
    print(results)

    # LR model training
    lr = LogisticRegression(random_state=0, max_iter=10e5, n_jobs=-1)
    lr.fit(X, y)
    lr_acc = lr.score(X_test, y_test)
    print("Logistic Reg acc: " + str(lr_acc))
    results[folder]["lr"] = str(lr_acc)
    dump(lr, outpath + "lr_model.joblib")
    print(results)

RF acc: 0.19894230579312253
{'all': {'rf': '0.19894230579312253'}}
[LibSVM]

Given these preliminary results, we then plotted the training accuracies for each model type (SVM, RF, LR), for each combination of protocols (HTTP, UDP, TCP, TLS).

In [ ]:
x_key = svm_key = list(map(lambda x : x[0], svm_results))
x_pos = [i for i, _ in enumerate(x_key)]

svm_results = [(k, r['svm']) for k,r in results.items()]
svm_val = list(map(lambda x : float(x[1]), svm_results))

rf_results = [(k, r['rf']) for k,r in results.items()]
rf_val = list(map(lambda x : float(x[1]), rf_results))

lr_results =  [(k, r['lr']) for k,r in results.items()]
lr_val = list(map(lambda x : float(x[1]), lr_results))

# multiple line plot
plt.plot(x_pos, svm_val, marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=2, label='SVM acc')
plt.plot(x_pos, rf_val, marker='o', markerfacecolor='red', markersize=12, color='olive', linewidth=2, label='RF acc')
plt.plot(x_pos, lr_val, marker='o', markerfacecolor='green', markersize=12, color='yellow', linewidth=2, label='LR acc')
plt.xticks(x_pos, x_key, rotation='vertical')
plt.xlabel("protocol")
plt.ylabel("Accuracy")
plt.title("Model accuracy of each protocol type on each model")

plt.legend()

Additionally, we also wanted to see what the average accuracy of the 3 model types are

In [ ]:
lr_av = np.mean(lr_val)
rf_av = np.mean(rf_val)
svm_av = np.mean(svm_val)
plt.bar([0,1,2], [lr_av, rf_av, svm_av])
plt.xticks([0,1,2], ['lr', 'rf', 'svm'])
plt.xlabel("Model")
plt.ylabel("Average accuracy")

plt.show()

In [ ]:
lr_av = np.mean(lr_val)
rf_av = np.mean(rf_val)
svm_av = np.mean(svm_val)
plt.bar([0,1,2], [lr_av, rf_av, svm_av])
plt.xticks([0,1,2], ['lr', 'rf', 'svm'])
plt.xlabel("Model")
plt.ylabel("Average accuracy")
plt.title("Average accuracy of the 3 model types")

plt.show()

As we can see from the plot above, using UDP with Logistical Regression (LR) seems to provide the best results. 

However, averaging out across protocols, LR performs the worst. 

We also want to verify the impact on accuracy depending on the protocol used. 

To do this we calculate the average accuracies for each model type (lr, rf, svm) for each model that uses http, udp, tls, tcp protocols. 

For example, we calculate the average accuracy given model type SVM that uses UDP packets (this includes the following data sets: UDP, HTTP + UDP, TCP + UDP, UDP + TCO + TLS, etc.)

In [ ]:
# For HTTP, TCP, UDP, TLS, plot their average accuracy on each of the 3 models
# x-axis: protcols that uses http, tcp, udp, or tls
# y-axis average accuracy on for models that used that protocol
http = [d for d in x_key if d.find("http") >= 0]
udp = [d for d in x_key if d.find("udp") >= 0]
tls = [d for d in x_key if d.find("tls") >= 0]
tcp = [d for d in x_key if d.find("tcp") >= 0]

# print(http)
# print(udp)
# print(tls)
# print(tcp)

http_svm_val = [r['svm'] for k,r in results.items() if k in http]
udp_svm_val = [r['svm'] for k,r in results.items() if k in udp]
tls_svm_val = [r['svm'] for k,r in results.items() if k in tls]
tcp_svm_val = [r['svm'] for k,r in results.items() if k in tcp]

http_lr_val = [r['lr'] for k,r in results.items() if k in http]
udp_lr_val = [r['lr'] for k,r in results.items() if k in udp]
tls_lr_val = [r['lr'] for k,r in results.items() if k in tls]
tcp_lr_val = [r['lr'] for k,r in results.items() if k in tcp]

http_rf_val = [r['rf'] for k,r in results.items() if k in http]
udp_rf_val = [r['rf'] for k,r in results.items() if k in udp]
tls_rf_val = [r['rf'] for k,r in results.items() if k in tls]
tcp_rf_val = [r['rf'] for k,r in results.items() if k in tcp]

http_svm_av = np.mean(list(map(lambda x: float(x), http_svm_val)))
http_lr_av = np.mean(list(map(lambda x: float(x), http_lr_val)))
http_rf_av = np.mean(list(map(lambda x: float(x), http_rf_val)))

udp_svm_av = np.mean(list(map(lambda x: float(x), udp_svm_val)))
udp_lr_av = np.mean(list(map(lambda x: float(x), udp_lr_val)))
udp_rf_av = np.mean(list(map(lambda x: float(x), udp_rf_val)))

tls_svm_av = np.mean(list(map(lambda x: float(x), tls_svm_val)))
tls_lr_av = np.mean(list(map(lambda x: float(x), tls_lr_val)))
tls_rf_av = np.mean(list(map(lambda x: float(x), tls_rf_val)))

tcp_svm_av = np.mean(list(map(lambda x: float(x), tcp_svm_val)))
tcp_lr_av = np.mean(list(map(lambda x: float(x), tcp_lr_val)))
tcp_rf_av = np.mean(list(map(lambda x: float(x), tcp_rf_val)))


from IPython.display import HTML, display
import tabulate
table = [["","SVM","LR", "RF"],
         ["HTTP", http_svm_av, http_lr_av, http_rf_av],
         ["UDP", udp_svm_av, udp_lr_av, udp_rf_av],
         ["TLS", tls_svm_av, tls_lr_av, tls_rf_av],
         ["TCP",tcp_svm_av, tcp_lr_av, tcp_rf_av]
        ]

print("Average accuracy of protocol usage against model")
display(HTML(tabulate.tabulate(table, tablefmt='html', floatfmt=(".4f", ".4f", ".4f"))))

# We would also want to normalize it by the average proportion of protocol packtes in each dataset that the protocol was part of

From the table above, it seems like HTTP and UDP plays a big role in SVM. 

However, findings in this table must be taken with a grain of salt as protocol usage is very skewed in the data i.e. a data collection might have 4 HTTP packets and hundreds of UDP.



At this point, UDP seems to be the most promising. We theorize that TCP data has too much noise where a lot of different services are also using TCP. Additionally, we guess that most of the UDP traffic are responsible by websites. Hence manual selection of the UDP protocol data provides the best results as it minimizes other noise. 

## Local data + Features

Since we decided to use HTTP + UDP packets for our local data training, we import them below

In [ ]:
# Local data
path = "collection/"
folder = "df_http_udp/" # specify which packet protocol config you want here
col_names = ["website-index", "time", "direction", "packet size"]
df_one_local = pd.read_csv(path+folder+"Day1-parsed-ondevice.csv", header=0, names=col_names, error_bad_lines=False, warn_bad_lines=False, quotechar='^')
df_two_local = pd.read_csv(path+folder+"Day2-parsed-ondevice.csv", header=0, names=col_names, error_bad_lines=False, warn_bad_lines=False, quotechar='^')
df_three_local = pd.read_csv(path+folder+"Day3-parsed-ondevice.csv", header=0, names=col_names, error_bad_lines=False, warn_bad_lines=False, quotechar='^')

We then proceeded with feature engineering. We chose the following features: 

- Cumulative representation of trace
- Cumulative Counting Statistics
    - Counting cumulative packets
    - Counting incoming cmulative packets 
    - Counting outgoing cumulative packets 
    - Counting cumulative percentage of outgoing/incoming packets 
- Previous 100 packet sizes
- Total Counting statistics
- Packet ordering statistics 
- Concentration of incoming & outgoing packets in first and last 30 packets 
- Concentration of outgoing packets 

All of the code for feature engineering can be found at the bottom of this document, along with their descriptions.

We do howerver, include a plot here of cumulative model accuracy (Random Forest) for each feature mentioned above

In [ ]:
x_key = ['cumulative representation', 'cumulative counting statistics', 'previous 100 packet sizes', 'total counting statistics', 'packet ordering statistics', 'concentration of incoming/outgoing in first/last 30', 'concentration of outgoing packets']
x_pos = [i for i, _ in enumerate(x_key)]
y_val = [0.7765592186406214, 0.7963265442181406, 0.813893796459882, 0.8336277875929198, 0.8277609253641788, 0.8339611320377346, 0.8328277609253641]

# multiple line plot
plt.plot(x_pos, y_val, marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=2, label='SVM acc')

plt.xticks(x_pos, x_key, rotation='vertical')
plt.xlabel("Feature")
plt.ylabel("Cumulative Accuracy")

## Training and testing local data 

In [ ]:
df_one_stat = outgoing_concentration(df_one, prev=True)
df_two_stat = outgoing_concentration(df_two, prev=True)
df_three_stat = outgoing_concentration(df_three, prev=True)

#### Panchenko et al [fingerprinting-ndss2016]; cumulative representation of trace

In [ ]:
def directional_packet_size(x):
    if (x['direction'] == 1):
        return -x['packet size']
    else:
        return x['packet size']

In [ ]:
def cum_trace(df):    
    # Create a directional packet size column p_i (negative values if outbound)
    df.insert(4, 'p_i', df['packet size'])
    df['p_i'] = df.apply(directional_packet_size, axis=1)

    # Create cumulative packet size trace using directional packet sizes
    df['c_i'] = df.groupby('website-index')['p_i'].cumsum(axis=None)

    return df

#### Statistics features per Hayes et al [sec16]

In [ ]:
def cumulative_packets(df, trace=False):
    # also use ndss2016
    if (trace):
        df = cum_trace(df)
        
    # cumulative packets counter
    df['cum_count'] = df.groupby('website-index').cumcount()+1
    
    # cumulative incoming packets counter
    df['cum_inc'] = df.groupby('website-index')['direction'].cumsum(axis=None)
    
    # cumulative outgoing packets counter
    df['cum_out'] = df['cum_count'] - df['cum_inc']
    
    # cumulative percentage outgoing/incoming
    df['perc_in'] = df['cum_inc']/(df.groupby('website-index').cumcount()+1)
    df['perc_out'] =  df['cum_out']/(df.groupby('website-index').cumcount()+1)
    
    return df

#### Previous 100 packet sizes as features

In [ ]:
def previous_packets(df, prev=False):
    if (prev):
        df = cumulative_packets(df, True)
    
    # Get 100 previous packet sizes --> we can make it p_i instead of packet_size too
    for i in range(1, 101):
        name = "prev_"+str(i)

        if (prev):
            df[name] = df.groupby('website-index')['p_i'].shift(i)
        else:
            df[name] = df.groupby('website-index')['packet size'].shift(i)
        
    # Remove NaN
    df.fillna(0, inplace=True)
    
    return df

#### Total discrete statistics 

Number of packet statistics, such as packet count per website, etc. Denoted in Hayes 2016

In [ ]:
def safe_div(q, r):
    if r == 0:
        return 0
    else:
        return q / r

def n_o_statistics(df, prev=False):
    if (prev):
        df = previous_packets(df, True)
    
    # Total number of packets per session, total number of incoming/outgoing packets + percentages
    curr_index = df.at[0, 'website-index']
    total_inc_packets = 0
    total_out_packets = 0
    inc_values = []
    out_values = []
    total_values = []
    total_incoming_percentage = []
    total_outgoing_percentage = []
    for index, row in df.iterrows():
        if (row['website-index'] != curr_index):
            total_packets = total_inc_packets + total_out_packets
            
            inc_values.extend([total_inc_packets] * total_packets)
            out_values.extend([total_out_packets] * total_packets)
            total_values.extend([total_packets] * total_packets)
            
            total_incoming_percentage.extend([safe_div(total_inc_packets, total_packets)] * (total_packets))
            total_outgoing_percentage.extend([safe_div(total_out_packets, total_packets)] * (total_packets))
            curr_index = row['website-index']
            
            if row['direction'] == 0:
                total_inc_packets = 1
                total_out_packets = 0
            else:
                total_inc_packets = 0
                total_out_packets = 1
        else: 
            if row['direction'] == 0:
                total_inc_packets += 1
            else:
                total_out_packets += 1
    # Repeat for last website index
    total_packets = total_inc_packets + total_out_packets
    inc_values.extend([total_inc_packets] * total_packets)
    out_values.extend([total_out_packets] * total_packets)
    total_values.extend([total_packets] * (total_packets))
    total_incoming_percentage.extend([safe_div(total_inc_packets, total_packets)] * (total_packets))
    total_outgoing_percentage.extend([safe_div(total_out_packets, total_packets)] * (total_packets))

    df['total_incoming'] = inc_values
    df['total_outgoing'] = out_values
    df['total_packets'] = total_values
    df['total_incoming_percentage'] = total_incoming_percentage
    df['total_outgoing_percentage'] = total_outgoing_percentage

    return df

#### Packet ordering statistics hayes 2016
For each successive incoming and outgoing packet, the total number of packets seen before it in the sequence

The standard deviation of the outgoing packet ordering list

The standard deviation of the incoming packet ordering list 

The average of the incoming packet ordering list


In [ ]:
def ordering_statistics(df, prev=False):
    if (prev):
        df = n_o_statistics(df, True)
    
    # For each successive incoming and outgoing packet, the total number of packets seen before it in the sequence
    """
    e.g. 
    direction successive_num_packets
    0 0 
    0 1
    0 2
    1 0 
    0 0
    0 1
    """
    total_seen = 0
    last_dir = -1
    result = []
    curr_index = ""
    for index, row in df.iterrows():
        if row['website-index'] != curr_index:
            total_seen = 0
            last_dir = -1
            curr_index = row['website-index']
        if row.direction == last_dir:
            total_seen += 1
            result.append(total_seen)
        else:
            total_seen = 0
            result.append(total_seen)
            last_dir = row.direction 
    df['successive_num_packets'] = result
    
    # The average and standard deviation of the in/outgoing packet ordering list
    curr_index = df.at[0, 'website-index']
    outgoing_vals = []
    incoming_vals = []
    incoming_order_std = []
    outgoing_order_std = []
    outgoing_order_avg = []
    incoming_order_avg = []

    for index, row in df.iterrows():
        if (row['website-index'] != curr_index):
            # Get std of in/outgoing_vals and reset
            curr_index = row['website-index']
            out_std = np.std(outgoing_vals)
            in_std = np.std(incoming_vals)
            out_avg = np.mean(outgoing_vals)
            in_avg = np.mean(incoming_vals)
            total_segment_length = len(outgoing_vals) + len(incoming_vals)
            incoming_order_std.extend([in_std] * total_segment_length)
            outgoing_order_std.extend([out_std] * total_segment_length)
            outgoing_order_avg.extend([out_avg] * total_segment_length)
            incoming_order_avg.extend([in_avg] * total_segment_length)
            
            if row.direction == 1:
                outgoing_vals = [row['successive_num_packets']]
                incoming_vals = []
            else: 
                incoming_vals = [(row['successive_num_packets']) ]
                outgoing_vals = []

        else:
            if row.direction == 1:
                outgoing_vals.append(row['successive_num_packets'])
            else: 
                incoming_vals.append(row['successive_num_packets']) 
                
    # Repeat for last segment 
    out_std = np.std(outgoing_vals)
    in_std = np.std(incoming_vals)
    out_avg = np.mean(outgoing_vals)
    in_avg = np.mean(incoming_vals)
    total_segment_length = len(outgoing_vals) + len(incoming_vals)
    incoming_order_std.extend([in_std] * total_segment_length)
    outgoing_order_std.extend([out_std] * total_segment_length)
    outgoing_order_avg.extend([out_avg] * total_segment_length)
    incoming_order_avg.extend([in_avg] * total_segment_length)
    
    
    df['outgoing_order_std'] = outgoing_order_std
    df['incoming_order_std'] = incoming_order_std
    df['outgoing_order_avg'] = outgoing_order_avg
    df['incoming_order_avg'] = incoming_order_avg

        
    return df

#### Concentration of incoming & outgoing packets in first & last 30 packets. Hayes 2016

The number of incoming and outgoing packets in the first and last 30 packets

In [ ]:
def first_last_30(df, prev=False):
    if (prev):
        df = ordering_statistics(df, True)
        
    # Number of incoming/outgoing packets in the first/last 30 packets 
    curr_index = df.at[0, 'website-index']
    num_inc_first_30_vals = []
    num_out_first_30_vals = []
    num_inc_last_30_vals = []
    num_out_last_30_vals = []
    dirs = []
    # Loop through each segment, grab direction column, take first 30, last 30, sum for in/out 
    for index, row in df.iterrows():
        if (row['website-index'] != curr_index):
            curr_index = row['website-index']
            # Sum up dirs
            ins_first_30 = len(list(filter(lambda d: d == 0, dirs[:30])))
            ins_last_30 = len(list(filter(lambda d: d == 0, dirs[-30:])))
            out_first_30 = len(list(filter(lambda d: d == 1, dirs[:30])))
            out_last_30 = len(list(filter(lambda d: d == 1, dirs[-30:])))
            num_packets = len(dirs)
            num_inc_first_30_vals.extend([ins_first_30] * num_packets)
            num_out_first_30_vals.extend([out_first_30] * num_packets)
            num_inc_last_30_vals.extend([ins_last_30] * num_packets)
            num_out_last_30_vals.extend([out_last_30] * num_packets)

            dirs = [row['direction']]
        else:
            dirs.append(row['direction'])

    ins_first_30 = len(list(filter(lambda d: d == 0, dirs[:30])))
    ins_last_30 = len(list(filter(lambda d: d == 0, dirs[-30:])))
    out_first_30 = len(list(filter(lambda d: d == 1, dirs[:30])))
    out_last_30 = len(list(filter(lambda d: d == 1, dirs[-30:])))
    num_packets = len(dirs)
    num_inc_first_30_vals.extend([ins_first_30] * num_packets)
    num_out_first_30_vals.extend([out_first_30] * num_packets)
    num_inc_last_30_vals.extend([ins_last_30] * num_packets)
    num_out_last_30_vals.extend([out_last_30] * num_packets)

    df['num_inc_first_30_vals'] = num_inc_first_30_vals
    df['num_out_first_30_vals'] = num_out_first_30_vals
    df['num_inc_last_30_vals'] = num_inc_last_30_vals
    df['num_out_last_30_vals'] = num_out_last_30_vals
  
    return df

#### Concentration of outgoing packets

The packet sequence split into non-overlapping chunks of 20 packets, where we count the number of outgoing packets in each chunk. Then extract the standard deviation and mean

In [ ]:
def get_segments(df):
    ret = []
    start = 0
    curr_index = df.at[0, 'website-index']
    for index, row in df.iterrows():
        if (row['website-index'] != curr_index):
            curr_index = row['website-index']
            ret.append((start, index - 1))
            start = index
            
    ret.append((start, index))

    return ret

def get_20s_from_segments(segments):
    # Given an array of start:end tuples, return an array of start2:end2 tuples where end2-start2 == 20, or less if not possible
    # e.g. [(0,45)] => [(0, 19), (20, 39), (40, 45)]
    ret = []
    for s in segments:
        start = s[0]
        end = s[1]
        i = start
        while i + 19 <= end:
            ret.append((i, i + 19))
            i += 20
        ret.append((i, end))
    return ret       


In [ ]:
def outgoing_concentration(df, prev=False):
    if (prev):
        df = first_last_30(df, True)
        
    # Number of outgoing packets in each chunk of 20 packets per segment
    # get an array of indexes for each segment 
    segments = get_segments(df)
    segment_20s = get_20s_from_segments(segments)    
    num_outgoing_vals = [] # Don't add this to the data, we only need it to collect statistics
#     debug = []
    for s in segment_20s:
        start = s[0]
        end = s[1]
        num_outs = df.iloc[start:end+1][df.direction == 1].shape[0]
        num_outgoing_vals.append(num_outs)
#         debug.extend([num_outs] * (end - start + 1))

        
    # We want to get the standard deviation, mean of the num_outgoing_vals for each segment
    # For each segment in segments, we want to obtain the corresponding values in num_outgoing_vals 
    # which are segmentized by 20s
    outgoing_20_std = []
    outgoing_20_avg = []

    for segment in segments:
        start = segment[0]
        end = segment[1]
        values = []
        for i in range(len(segment_20s)):
            seg20 = segment_20s[i]
            if seg20[0] < start:
                continue
            if seg20[0] > end:
                break
            values.append(num_outgoing_vals[i])
        mean = np.mean(values)
        std = np.std(values)
        outgoing_20_avg.extend([mean] * (end - start + 1))
        outgoing_20_std.extend([std] * (end - start + 1))

    df['outgoing_20_avg'] = outgoing_20_avg
    df['outgoing_20_std'] = outgoing_20_std
#     df['debug'] = debug

        
    return df

### Model execution

In [ ]:
def run_model(df_one, df_two, df_three):
    # Preprocess training data
    df = df_one.append(df_two)
    X = df.loc[:, df.columns != 'website-index']
    y = df['website-index']

    # Preprocess testing data
    X_test = df_three.loc[:, df_three.columns != 'website-index']
    y_test = df_three['website-index']

    # Model training
    clf = RandomForestClassifier(n_estimators=300, n_jobs=-1 )
    # Change n_estimators to what you want (defualt 100)
    # n_jobs=-1 will run the classifier on ALL cpu's, lower this if running on bad comp
    
    clf.fit(X,y)

    knn = KNeighborsClassifier(n_neighbors=20)
    knn.fit(X,y)

    # Evaluate
    rf_acc = clf.score(X_test, y_test)
    knn_acc = knn.score(X_test, y_test)

    print("RF acc: " + str(rf_acc))
    print("KNN acc: " + str(knn_acc))